https://python.langchain.com/docs/modules/agents/agent_types/react

In [1]:
from dotenv import load_dotenv, find_dotenv; load_dotenv(find_dotenv())

True

In [2]:
!export LANGCHAIN_TRACING_V2=true

In [3]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI

In [4]:
tools = [TavilySearchResults(max_results=3)]

In [5]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/react")
prompt

/Users/harukary/Documents/workspace/llm_samples/.venv/lib/python3.10/site-packages/langchain_core/_api/beta_decorator.py:160: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


PromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}')

In [6]:
# Choose the LLM to use
llm = ChatOpenAI(model='gpt-3.5-turbo-16k')

# Construct the ReAct agent
agent = create_react_agent(llm, tools, prompt)

In [7]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(
    agent=agent, tools=tools,
    handle_parsing_errors=True,
    verbose=True
)

In [8]:
agent_executor.invoke({"input": "what is LangChain?"})



> Entering new AgentExecutor chain...
I need to search for information about LangChain to answer the question accurately.
Action: tavily_search_results_json
Action Input: "LangChain"[{'url': 'https://blog.stackademic.com/langchain-in-chains-1-a-closer-look-a6bbaa246ccb', 'content': 'Langchain  LangChain is an open-source framework designed to simplify the creation of applications using large language models  Langchain provides various components that make it easier to integrate and manage models in different application  LangChain is a framework for developing applications powered by language models. It enables applications that:1 day ago -- This is the first part of a series about LangChain. I will introduce its core concepts. In each subsequent post, we will expand our understanding of the framework. LangChain is an open-source framework designed to simplify the creation of applications using large language models (LLMs).'}, {'url': 'https://medium.com/@okanyenigun/langchain-in-cha

{'input': 'what is LangChain?',
 'output': 'LangChain is an open-source framework for developing applications powered by language models.'}

### Using with chat history

In [9]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/react-chat")
prompt

/Users/harukary/Documents/workspace/llm_samples/.venv/lib/python3.10/site-packages/langchain_core/_api/beta_decorator.py:160: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


PromptTemplate(input_variables=['agent_scratchpad', 'chat_history', 'input', 'tool_names', 'tools'], template='Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, Assistant is able to generate its own text based on the input it receives, allowing it to engage in discussions and provide explanations and descr

In [10]:
# Construct the ReAct agent
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent, tools=tools,
    handle_parsing_errors=True,
    verbose=True
)

In [11]:
from langchain_core.messages import AIMessage, HumanMessage

agent_executor.invoke(
    {
        "input": "My hometown is Seto. What is the famous thing in there? Only use a tool if needed, otherwise respond with Final Answer",
        # Notice that chat_history is a string, since this prompt is aimed at LLMs, not chat models
        "chat_history": "Human: Hi! My name is Bob\nAI: Hello Bob! Nice to meet you",
    }
)



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: tavily_search_results_json
Action Input: "famous things in Seto"[{'url': 'https://travopo.com/travel-guides/japan/seto-shi/', 'content': "places to visit and things to do in Seto shi in 2024. Make your own Itinerary, like a Pro.  Fun Things to Do in Seto shi | Travel Guide (2024) | Best Places to Visit  Here are some things to do in Seto-shi:  4. Experience the Seto International Ceramic and Glass Art Exchange Exhibition: Seto-shi is famous for its ceramics,1. Visit the Seto City Art Museum: This museum is renowned for its collection of contemporary art, including works by local ceramic artists. Explore the various exhibitions and gain insight into the city's artistic legacy. 2."}, {'url': 'https://www.morningstar.com/news/globe-newswire/9011755/seven-secret-places-in-setouchi-to-visit-in-2024', 'content': "Seven Secret Places in Setouchi to Visit in 2024 Setouchi, Japan, Jan. 03, 2024  So Much More t

{'input': 'My hometown is Seto. What is the famous thing in there? Only use a tool if needed, otherwise respond with Final Answer',
 'chat_history': 'Human: Hi! My name is Bob\nAI: Hello Bob! Nice to meet you',
 'output': "Seto is famous for its ceramics and art. The Seto International Ceramic and Glass Art Exchange Exhibition is a popular attraction, showcasing the city's renowned ceramics. The Seto City Art Museum is also worth visiting, as it houses a collection of contemporary art, including works by local ceramic artists. Additionally, Seto is located near Naoshima, an art island in the Seto Inland Sea known for its modern museums and galleries."}